In [68]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, TensorDataset

In [69]:
data = pd.read_csv('climatedatawithdate.csv')  # Replace 'your_dataset.csv' with your actual dataset file path

In [53]:
data['date'] = pd.to_datetime(data['date'])
# data.set_index('date', inplace=True)

In [54]:
conditions = data[['tauu', 'rlus', 'sfcWind', 'rsuscs', 'rsus', 'rsdscs', 'prw', 'hurs', 'rsuscs', 'rsus', 'rsdscs', 'prw', 'hurs', 'rtmt',
                   'ua_850', 'evspsbl', 'vas', 'clwvi', 'rsutcs', 'rsds', 'ua_500', 'va_850', 'pr', 'huss', 'sbl', 'zg_500', 'wap_500', 'clt', 
                   'va_500', 'clivi', 'ta_500', 'psl', 'hfls', 'hfss', 'prc', 'ps', 'rlds', 'rldcs','rlut', 'rlutcs', 'rsdt', 'rsut', 'ta_850', 
                   'tas', 'tasmax', 'tasmin', 'tauv', 'tro3_500', 'tro3_850', 'ts', 'uas', 'wap_850', 'zg_850']]  # Replace 'condition1', 'condition2', etc. with your actual condition columns
precipitation = data['puttalam']


In [40]:
class TimeSeriesDataset(Dataset):
    def __init__(self, conditions, precipitation):
        self.conditions = conditions
        self.precipitation = precipitation

    def __len__(self):
        return len(self.conditions)

    def __getitem__(self, index):
        x = self.conditions[index]
        y = self.precipitation[index]
        return x, y

In [55]:
conditions_train, conditions_test, precipitation_train, precipitation_test = train_test_split(conditions, precipitation, test_size=0.2, shuffle=False)


In [41]:
dataset_train = TimeSeriesDataset(conditions_train, precipitation_train)

# Set batch_size and other relevant parameters
batch_size = 32
shuffle = True

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=shuffle)

dataset_test = TimeSeriesDataset(conditions_test, precipitation_test)

# Set batch_size and other relevant parameters
batch_size = 32
shuffle = True

data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=shuffle)


In [56]:
conditions_train = torch.Tensor(conditions_train.values)
conditions_test = torch.Tensor(conditions_test.values)
precipitation_train = torch.Tensor(precipitation_train.values)
precipitation_test = torch.Tensor(precipitation_test.values)


In [64]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = conditions_train.shape[1]  # Adjust based on the number of input features
hidden_size = 64  # Adjust the hidden size according to your preference
output_size = 1  # Adjust based on the number of output classes or variables

model = LSTMModel(input_size, hidden_size, output_size)


In [65]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [66]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    optimizer.zero_grad()

    # Forward pass
    outputs = model(conditions_train)
    loss = criterion(outputs, precipitation_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Print the loss for monitoring
    print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')

IndexError: too many indices for tensor of dimension 2

In [67]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    optimizer.zero_grad()

    # Forward pass
    outputs = model(conditions_train)
    loss = criterion(outputs, precipitation_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Print the loss for monitoring
    print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')


IndexError: too many indices for tensor of dimension 2

In [72]:
# Step 3: Split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

# Step 4: Prepare the data for LSTM
def prepare_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size):
        sequence = data[i:i+window_size+1]
        sequences.append(sequence)
    return sequences

window_size = 10  # Adjust the window size based on your requirements

train_sequences = prepare_sequences(train_data, window_size)
test_sequences = prepare_sequences(test_data, window_size)

In [81]:
# Step 5: Convert the data to PyTorch tensors
def sequences_to_tensors(sequences):
    X = torch.tensor([seq[:-1] for seq in sequences], dtype=torch.float32)
    y = torch.tensor([seq[-1][-1] for seq in sequences], dtype=torch.float32)
    return X, y

X_train, y_train = sequences_to_tensors(train_data)
X_test, y_test = sequences_to_tensors(test_data)

ValueError: too many dimensions 'str'

In [ ]:


# Step 5: Convert the data to PyTorch tensors
def sequences_to_tensors(sequences):
    X = torch.tensor([seq[:-1] for seq in sequences], dtype=torch.float32)
    y = torch.tensor([seq[-1][-1] for seq in sequences], dtype=torch.float32)
    return X, y

X_train, y_train = sequences_to_tensors(train_sequences)
X_test, y_test = sequences_to_tensors(test_sequences)

# Step 6: Create a custom LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden[-1])
        return output

input_size = X_train.shape[2]
hidden_size = 64  # Adjust the hidden size based on your requirements
output_size = 1

model = LSTMModel(input_size, hidden_size, output_size)

# Step 7: Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 8: Train the LSTM model
num_epochs = 10

for epoch in range(num_epochs):
    optimizer.zero_grad()

    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)
    
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Step 9: Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs.squeeze(), y_test)
    print(f"Test Loss: {test_loss.item()}")

# Step 10: Analyze the results (calculate evaluation metrics, visualize predictions, etc.)


In [78]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [79]:
print(data.dtypes)

puttalam    float64
tauu        float64
rlus        float64
sfcWind     float64
rsuscs      float64
rsus        float64
rsdscs      float64
prw         float64
hurs        float64
rtmt        float64
ua_850      float64
evspsbl     float64
vas         float64
clwvi       float64
rsutcs      float64
rsds        float64
ua_500      float64
va_850      float64
pr          float64
huss        float64
sbl         float64
zg_500      float64
wap_500     float64
clt         float64
va_500      float64
clivi       float64
ta_500      float64
psl         float64
hfls        float64
hfss        float64
prc         float64
ps          float64
rlds        float64
rldcs       float64
rlut        float64
rlutcs      float64
rsdt        float64
rsut        float64
ta_850      float64
tas         float64
tasmax      float64
tasmin      float64
tauv        float64
tro3_500    float64
tro3_850    float64
ts          float64
uas         float64
wap_850     float64
zg_850      float64
dtype: object
